In [2]:
# !pip install wandb
# !pip install transformers
# !pip install sentencepiece

import pandas as pd
import numpy as np
import re
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import math
import torch.optim as optim
import os
import wandb

# wandb.login()

os.environ["WANDB_DISABLED"] = "true"

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached promise-2.3.tar.gz (19 kB)
  Using cached gitdb-4.0.9-py3-none-any.whl (63 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=f6ca6715c9124b9cc712e5e84ee602eeb44540a8d57caa5469e81b43d70a4a37
  Stored in directory: c:\users\hadye\appdata\local\pip\cache\wheels\e1\e8\83\ddea66100678d139b14bc87692ece57c6a2a937956d2532608
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=178b217146dedc312b1ccc9eca9d8dd7a5ca6f644dad3cc5ebf6f27a39d23b77
  Stored in directory: c:\users\hadye\appdata\local\pip\cache\wheels\b7\0a\67\ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built promise pathtools


In [3]:
# Create torch dataset
# https://towardsdatascience.com/fine-tuning-pretrained-nlp-models-with-huggingfaces-trainer-6326a4456e7b

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels['input_ids'][idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [4]:
max_source_length = 512
max_target_length = 128

In [19]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration 
from transformers import EarlyStoppingCallback

tokenizer = T5TokenizerFast.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)

In [7]:
# training_sample = training_data.sample(frac=0.4, random_state=1)

training_data = pd.read_csv('./1-NER_Data/1-csv_format/train/training_data.csv')
training_data

,input_text,target_text
0,EU rejects German call to boycott British lamb .,"*EU*,*German*,*British*"
1,Peter Blackburn,*Peter Blackburn*
2,BRUSSELS 1996-08-22,*BRUSSELS*
3,The European Commission said on Thursday it di...,"*European Commission*,*German*,*British*"
4,Germany 's representative to the European Unio...,"*Germany*,*European Union*,*Werner Zwingmann*,..."
...,...,...
159090,""" @Mackenzie_71 : Super Bowl Sunday """,**
159091,Watch Barrow - Boston United Live December 20 ...,"*Barrow*,*Boston United*"
159092,"On January 22nd , from 11- 1.30 , TVPS will be...",*TVPS*
159093,"Conditioning plan all made up for tomorrow , f...",**


In [10]:
X_train_tokenized = tokenizer(['ner: ' + sequence for sequence in training_data["input_text"]], 
                              padding=True, 
                              truncation=True, 
                              max_length=max_source_length)

y_train_tokenized = tokenizer(list(training_data["target_text"]), 
                              padding=True, 
                              truncation=True, 
                              max_length=max_target_length)

print(len(training_data))
# print(len(training_sample))

159095


In [14]:
validation_data = pd.read_csv('./1-NER_Data/1-csv_format/val/val_data.csv')
validation_data

,input_text,target_text,word_count,NE_count,%_NE_in_sentence
0,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,*LEICESTERSHIRE*,9,1,0.1
1,LONDON 1996-08-30,*LONDON*,2,1,0.5
2,West Indian all-rounder Phil Simmons took four...,"*West Indian*,*Phil Simmons*,*Leicestershire*,...",34,6,0.2
3,"Their stay on top , though , may be short-live...","*Essex*,*Derbyshire*,*Surrey*,*Kent*,*Nottingh...",33,5,0.2
4,After bowling Somerset out for 83 on the openi...,"*Somerset*,*Grace Road*,*Leicestershire*,*Engl...",36,7,0.2
...,...,...,...,...,...
26573,you sorted that Amos thing mom,*Amos*,6,1,0.2
26574,He used chi blast to drop that old man . ( tha...,*the invisible man*,47,3,0.1
26575,your sarcasm is goals : D : D,**,8,0,0.0
26576,excellent poem Dubbz﻿,*Dubbz﻿*,3,1,0.3


In [15]:
X_val_tokenized = tokenizer(['ner: ' + sequence for sequence in validation_data["input_text"]], 
                              padding=True, 
                              truncation=True, 
                              max_length=max_source_length)

y_val_tokenized = tokenizer(list(validation_data["target_text"]), 
                              padding=True, 
                              truncation=True, 
                              max_length=max_target_length)

print(len(validation_data))
# print(len(training_sample))

26578


In [11]:
train_dataset = Dataset(X_train_tokenized, y_train_tokenized)

In [16]:
val_dataset = Dataset(X_val_tokenized, y_val_tokenized)

In [21]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    "test_trainer",
    evaluation_strategy ='steps',
    eval_steps = 50, # Evaluation and Save happens every 50 steps
    save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    learning_rate = 1e-3,
    adam_epsilon = 1e-8,
    num_train_epochs = 5,
#     report_to="wandb",
    metric_for_best_model = 'f1',
    load_best_model_at_end=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [22]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model, 
    args=training_args, 
    train_dataset= train_dataset,
    eval_dataset = val_dataset,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [23]:
trainer.train()

C:\Users\hadye\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 159095
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 795475


Step,Training Loss,Validation Loss


KeyboardInterrupt: 